In [60]:
import torch
from torch import nn as nn
from torch import tensor
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
print(torch.__version__)


2.3.1+cpu


Creating our Model

In [61]:
class BinaryModel(nn.Module):
    def __init__(self):
        super(BinaryModel, self).__init__()
        self.linear1 = nn.Linear(9,7)
        self.linear2 = nn.Linear(7,4)
        self.linear3 = nn.Linear(4,1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.activation(x)
        return x
    
model = BinaryModel()

Pre-Processing Data

In [62]:
data = pd.read_csv('./machine_failure.csv')
print(data.isnull().sum())
features = data.drop('fail', axis=1)
labels = data['fail']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
training_features, testing_features, training_labels, testing_labels = train_test_split(scaled_features, labels, train_size=0.2)
training_features = tensor(training_features, dtype=torch.float32)
training_labels = tensor(training_labels.values, dtype=torch.float32)
testing_features = tensor(testing_features, dtype=torch.float32)
testing_labels = tensor(testing_labels.values, dtype=torch.float32)

footfall       0
tempMode       0
AQ             0
USS            0
CS             0
VOC            0
RP             0
IP             0
Temperature    0
fail           0
dtype: int64


Data Loaders
feed the model tensors and provide labels for evaluation both during testing and training process.

In [63]:
from torch.utils.data import DataLoader, TensorDataset

training_dataset = TensorDataset(training_features, training_labels)
testing_dataset = TensorDataset(testing_features, testing_labels)

training_loader = DataLoader(training_dataset, batch_size=40, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=40, shuffle=False)

Starting the training loop

In [64]:
from torch import optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

model.train()
num_of_epochs = 2400
for epoch in range(num_of_epochs):
  for features, labels in training_loader:
    optimizer.zero_grad()
    output = model(features)
    loss = criterion(output, labels.view(-1,1))
    loss.backward()
    optimizer.step()
  if epoch%100 == 0:
    print(f'Epoch {epoch+1}/{num_of_epochs}, Loss: {loss.item()}')


Epoch 1/2400, Loss: 0.7644695043563843
Epoch 101/2400, Loss: 0.2538984417915344
Epoch 201/2400, Loss: 0.25200706720352173
Epoch 301/2400, Loss: 0.33827465772628784
Epoch 401/2400, Loss: 0.08609678596258163
Epoch 501/2400, Loss: 0.17609095573425293
Epoch 601/2400, Loss: 0.13988488912582397
Epoch 701/2400, Loss: 0.36960864067077637
Epoch 801/2400, Loss: 0.11546728760004044
Epoch 901/2400, Loss: 0.10772253572940826
Epoch 1001/2400, Loss: 0.06184583157300949
Epoch 1101/2400, Loss: 0.10436626523733139
Epoch 1201/2400, Loss: 0.24191156029701233
Epoch 1301/2400, Loss: 0.2548418641090393
Epoch 1401/2400, Loss: 0.29484400153160095
Epoch 1501/2400, Loss: 0.21164098381996155
Epoch 1601/2400, Loss: 0.08300195634365082
Epoch 1701/2400, Loss: 0.17594607174396515
Epoch 1801/2400, Loss: 0.12987369298934937
Epoch 1901/2400, Loss: 0.2864238917827606
Epoch 2001/2400, Loss: 0.15858721733093262
Epoch 2101/2400, Loss: 0.2946973443031311
Epoch 2201/2400, Loss: 0.11780799180269241
Epoch 2301/2400, Loss: 0.093

In [65]:
from torchmetrics import Accuracy 

accuracy = Accuracy(task='binary')
model.eval()
with torch.no_grad():
    for features, labels in testing_loader:
        output = model(features)
        prediction = output.round()
        accuracy.update(prediction, labels.view(-1,1))
        model.train()

        print(f'Accuracy: {accuracy.compute().item()}')

Accuracy: 0.925000011920929
Accuracy: 0.925000011920929
Accuracy: 0.8999999761581421
Accuracy: 0.8999999761581421
Accuracy: 0.8799999952316284
Accuracy: 0.8916666507720947
Accuracy: 0.8892857432365417
Accuracy: 0.8968750238418579
Accuracy: 0.8972222208976746
Accuracy: 0.9024999737739563
Accuracy: 0.9022727012634277
Accuracy: 0.9020833373069763
Accuracy: 0.9038461446762085
Accuracy: 0.9017857313156128
Accuracy: 0.903333306312561
Accuracy: 0.9046875238418579
Accuracy: 0.9058823585510254
Accuracy: 0.9055555462837219
Accuracy: 0.9060846567153931
